# Submission notebook

--------------------------

## Load libraries

In [13]:
import gc
import os
import random
import sys
import time
import warnings
warnings.simplefilter("ignore")

#import pdb
#import zipfile
#import pydicom
from albumentations import *
from albumentations.pytorch import ToTensor
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image, ImageFilter
import rasterio
from rasterio.windows import Window
from sklearn.model_selection import KFold
import tifffile as tiff
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset, sampler
from tqdm import tqdm_notebook as tqdm

In [14]:
import segmentation_models_pytorch as smp

##  Set parameters

In [15]:
def set_seed(seed=46):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
set_seed(46)

To save time, the number of "folds" is smaller. 
These days, depending on the model, I think it's more common to use 5 ~ 10.

In [35]:
fold = 0
nfolds = 5
reduce = 4
sz = 256

BATCH_SIZE = 2
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
NUM_WORKERS = 2
SEED = 2020
TH = 0.50  #threshold for positive predictions

DATA = 'test'
LABELS = 'train.csv'
df_sample = pd.read_csv('test.csv')

In [38]:
df_sample
#import numpy as np
df_sample['predicted'] = np.nan
df_sample

,id,predicted
0,CL_HandE_1234_B004_bottomleft,NaN
1,HandE_B005_CL_b_RGB_bottomleft,NaN


## Util functions

In [39]:

def rle_encode_less_memory(img):
    
    pixels = img.T.flatten()
    
    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

## Dataset

In [40]:
def get_aug(p=1.0):
    return Compose([
        HorizontalFlip(),
        VerticalFlip(),
        RandomRotate90(),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=0.9, 
                         border_mode=cv2.BORDER_REFLECT),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.3),
        OneOf([
            HueSaturationValue(10,15,10),
            CLAHE(clip_limit=2),
            RandomBrightnessContrast(),            
        ], p=0.3),
    ], p=p)

## Model

In [41]:
def get_UnetPlusPlus():
    model =  smp.UnetPlusPlus(
                 encoder_name='efficientnet-b3',
                 encoder_weights=None,
                 in_channels=3,
                 classes=1)
    return model

# Inference


In [42]:
mean = np.array([0.63701495, 0.4709702,  0.6817423] )
std = np.array([0.15978882, 0.2245109, 0.14173926])

def img2tensor(img,dtype:np.dtype=np.float32):
    if img.ndim==2 : img = np.expand_dims(img,2)
    img = np.transpose(img,(2,0,1))
    return torch.from_numpy(img.astype(dtype, copy=False))

class HuBMAPTestDataset(Dataset):
    def __init__(self, idx, sz=sz, reduce=reduce):
        self.data = rasterio.open(os.path.join(DATA,idx+'.tiff'), transform = identity,
                                 num_threads='all_cpus')
       
        if self.data.count != 3:
            subdatasets = self.data.subdatasets
            self.layers = []
            if len(subdatasets) > 0:
                for i, subdataset in enumerate(subdatasets, 0):
                    self.layers.append(rasterio.open(subdataset))
        self.shape = self.data.shape
        self.reduce = reduce
        self.sz = reduce*sz
        self.pad0 = (self.sz - self.shape[0]%self.sz)%self.sz
        self.pad1 = (self.sz - self.shape[1]%self.sz)%self.sz
        self.n0max = (self.shape[0] + self.pad0)//self.sz
        self.n1max = (self.shape[1] + self.pad1)//self.sz
        
    def __len__(self):
        return self.n0max*self.n1max
    
    def __getitem__(self, idx):
     
        n0,n1 = idx//self.n1max, idx%self.n1max
        
        
        x0,y0 = -self.pad0//2 + n0*self.sz, -self.pad1//2 + n1*self.sz
        # make sure that the region to read is within the image
        p00,p01 = max(0,x0), min(x0+self.sz,self.shape[0])
        p10,p11 = max(0,y0), min(y0+self.sz,self.shape[1])
        img = np.zeros((self.sz,self.sz,3),np.uint8)
        # mapping the loade region to the tile
        if self.data.count == 3:
            img[(p00-x0):(p01-x0),(p10-y0):(p11-y0)] = np.moveaxis(self.data.read([1,2,3],
                window=Window.from_slices((p00,p01),(p10,p11))), 0, -1)
        else:
            for i,layer in enumerate(self.layers):
                img[(p00-x0):(p01-x0),(p10-y0):(p11-y0),i] =\
                  layer.read(1,window=Window.from_slices((p00,p01),(p10,p11)))
        
        if self.reduce != 1:
            img = cv2.resize(img,(self.sz//reduce,self.sz//reduce),
                             interpolation = cv2.INTER_AREA)
        #check for empty imges
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        h,s,v = cv2.split(hsv)
        if (s>s_th).sum() <= p_th or img.sum() <= p_th:
            #images with -1 will be skipped
            return img2tensor((img/255.0 - mean)/std), -1
        else: return img2tensor((img/255.0 - mean)/std), idx

In [43]:
models = []

for fold in range(nfolds):

    model = get_UnetPlusPlus().to(DEVICE)
    model.load_state_dict(torch.load(f"./FOLD{fold}_.pth"))
    models.append(model)

In [44]:
#iterator like wrapper that returns predicted masks
class Model_pred:
    def __init__(self, models, dl, tta:bool=True, half:bool=False):
        self.models = models
        self.dl = dl
        self.tta = tta
        self.half = half
        
    def __iter__(self):
        count=0
        with torch.no_grad():
            for x,y in iter(self.dl):
                if ((y>=0).sum() > 0): #exclude empty images
                    x = x[y>=0].to(DEVICE)
                    y = y[y>=0]
                    if self.half: x = x.half()
                    py = None
                    for model in self.models:
                        p = model(x)
                        p = torch.sigmoid(p).detach()
                        if py is None: py = p
                        else: py += p
                    if self.tta:
                        
                        flips = [[-1],[-2],[-2,-1]]
                        for f in flips:
                            xf = torch.flip(x,f)
                            for model in self.models:
                                p = model(xf)
                                p = torch.flip(p,f)
                                py += torch.sigmoid(p).detach()
                        py /= (1+len(flips))        
                    py /= len(self.models)

                    py = F.upsample(py, scale_factor=reduce, mode="bilinear")
                    py = py.permute(0,2,3,1).float().cpu()
                    
                    batch_size = len(py)
                    for i in range(batch_size):
                        yield py[i],y[i]
                        count += 1
                    
    def __len__(self):
        return len(self.dl.dataset)

In [45]:
df_sample

,id,predicted
0,CL_HandE_1234_B004_bottomleft,NaN
1,HandE_B005_CL_b_RGB_bottomleft,NaN


In [46]:
LABELS = 'test.csv'
d = pd.read_csv(LABELS)
#d = d.drop(d[d.id == 'HandE_B005_CL_b_RGB_topright'].index[0])
ids = d.id.values
ids

array(['CL_HandE_1234_B004_bottomleft', 'HandE_B005_CL_b_RGB_bottomleft'],
      dtype=object)

In [47]:
s_th = 40  #saturation blancking threshold
p_th = 1000*(sz//256)**2 #threshold for the minimum number of pixels
identity = rasterio.Affine(1, 0, 0, 0, 1, 0)

names,preds = [],[]
print(df_sample)
for idx,row in tqdm(df_sample.iterrows(),total=len(df_sample)):
    idx = row['id']
    ds = HuBMAPTestDataset(idx)
    #rasterio cannot be used with multiple workers
    dl = DataLoader(ds,BATCH_SIZE,num_workers=0,shuffle=False,pin_memory=True)
    mp = Model_pred(models,dl)
    #generate masks
    mask = torch.zeros(len(ds),ds.sz,ds.sz,dtype=torch.int8)
    for p,i in iter(mp): mask[i.item()] = p.squeeze(-1) > TH
    
    #reshape tiled masks into a single mask and crop padding
    mask = mask.view(ds.n0max,ds.n1max,ds.sz,ds.sz).\
        permute(0,2,1,3).reshape(ds.n0max*ds.sz,ds.n1max*ds.sz)
    mask = mask[ds.pad0//2:-(ds.pad0-ds.pad0//2) if ds.pad0 > 0 else ds.n0max*ds.sz,
        ds.pad1//2:-(ds.pad1-ds.pad1//2) if ds.pad1 > 0 else ds.n1max*ds.sz]
    
    
    rle = rle_encode_less_memory(mask.numpy())
    names.append(idx)
    preds.append(rle)
    del mask, ds, dl
    gc.collect()

                               id  predicted
0   CL_HandE_1234_B004_bottomleft        NaN
1  HandE_B005_CL_b_RGB_bottomleft        NaN


  0%|          | 0/2 [00:00<?, ?it/s]

In [48]:
df = pd.DataFrame({'id':names,'predicted':preds})
print(df)
df.to_csv('submission_output_file.csv',index=False)

                               id  \
0   CL_HandE_1234_B004_bottomleft   
1  HandE_B005_CL_b_RGB_bottomleft   

                                           predicted  
0  416774 11 421305 20 425835 28 430365 36 434897...  
1  4053 50 8588 54 13122 57 17657 60 22192 63 267...  
